In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from DNN_module import Net
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

In [2]:
net0 = Net(D_in = 5, D_out = 3, H = 64, H2 = 64, H3 = 64)
net1 = Net(D_in = 5, D_out = 3, H = 256, H2 = 256, H3 = 256)
net2 = Net(D_in = 5, D_out = 3, H = 512, H2 = 512, H3 = 512)


def total_params(net):
    return sum(p.numel() for p in net.parameters() if p.requires_grad)

print(total_params(net0))
print(total_params(net1))
print(total_params(net2))

9283
135427
532995


In [3]:
%run NCoinDP_functions.ipynb

In [4]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu"
device = torch.device(dev) 

2.1.2+cu121
True
NVIDIA A10


In [ ]:
# Default : cuda
torch.set_default_device('cuda')

# Number of train and validation data
L_train = 100000
L_val   =  50000

# Define the batch size
BATCH_SIZE = 64

# Number of Epochs
N_EPOCHS = 250
    
# Number of Simulations
sims = 50

# net
nets = [64, 256, 512]

for sim in range(0, sims):
    tmp = "syn_data/OU_sim_n3000.pt"
    tmp = torch.load(tmp)

    # Data import
    [X, Y] = tmp
    X = X.to(device)
    Y = Y.to(device)

    torch.manual_seed(1000 + sim)
    indexes = torch.randperm(L_train + L_val)

    # Divide Data
    X_train = X[indexes[0:L_train]]
    Y_train = Y[indexes[0:L_train]]

    X_val = X[indexes[L_train:(L_train + L_val)]]
    Y_val = Y[indexes[L_train:(L_train + L_val)]]
    
    # Use torch.utils.data to create a DataLoader that will take care of creating batches
    dataset = TensorDataset(X_train, Y_train)
    dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, generator=torch.Generator(device=device))

    # Get the dataset size for printing (it is equal to N_SAMPLES)
    dataset_size = len(dataloader.dataset)

    # Define the input and output dimensions
    D_in, D_out = X_train.size()[1], Y_train.size()[1]


    for net_num in nets:
        # Create an instance of the Net class with specified dimensions
        net = Net(D_in = D_in, D_out = D_out, H = net_num, H2 = net_num, H3 =net_num)

        # Model name
        model_save_name = 'nets/SA/net'+str(net_num)+"_"+str(sim)+'.pt'
        path = F"./{model_save_name}"

        # The nn package also contains definitions of popular loss functions; in this case we will use Mean Squared Error (MSE) as our loss function.
        loss_fn = torch.nn.MSELoss(reduction='sum')
        learning_rate = 1e-4
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)

        train_error_plt = []
        val_error_plt = []

        torch.manual_seed(2000 + sim)

        # Loop over epochs
        for epoch in range(N_EPOCHS):
            for id_batch, (x_batch, y_batch) in enumerate(dataloader):
                y_batch_pred = net(x_batch)
                loss = loss_fn(y_batch_pred, y_batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if epoch % 50 ==0 and id_batch % 100 == 0:
                    loss, current = loss.item(), (id_batch + 1)* len(x_batch)
                    print(f"train_loss: {loss/BATCH_SIZE:>7f}  [{current:>5d}/{dataset_size:>5d}]")

            with torch.no_grad():
                   net.eval()
                   theta_pred_train = net(X_train)
                   train_loss = loss_fn(theta_pred_train,Y_train) / L_train
                   train_error_plt = np.append(train_error_plt, train_loss.to("cpu"))

                   theta_pred_val = net(X_val)
                   val_loss = loss_fn(Y_val, theta_pred_val) / L_val
                   val_error_plt = np.append(val_error_plt, val_loss.to("cpu"))

            if epoch % 10 ==0:
                print(f"Epoch {epoch + 1}\n-------------------------------")
                print(f"train_loss {train_loss:>7f} val_loss {val_loss:>7f}")
                print("learning rate: ", learning_rate, ", net num: ", net_num, ", sim: ", sim)

            ## Choose Best Model
            if val_error_plt[epoch] == np.min(val_error_plt):
                 best=epoch
                 torch.save(net.state_dict(), path)

            if epoch % 100 ==99:
                net.load_state_dict(torch.load(path))
                learning_rate = max(learning_rate * 1e-1, 1e-9)


train_loss: 15.286289  [   64/100000]
train_loss: 9.811146  [ 6464/100000]
train_loss: 9.239730  [12864/100000]
train_loss: 6.517861  [19264/100000]
train_loss: 4.120437  [25664/100000]
train_loss: 3.613735  [32064/100000]
train_loss: 2.958387  [38464/100000]
train_loss: 2.063457  [44864/100000]
train_loss: 1.852620  [51264/100000]
train_loss: 1.245010  [57664/100000]
train_loss: 0.856904  [64064/100000]
train_loss: 0.645236  [70464/100000]
train_loss: 0.373699  [76864/100000]
train_loss: 0.368848  [83264/100000]
train_loss: 0.357799  [89664/100000]
train_loss: 0.215923  [96064/100000]
Epoch 1
-------------------------------
train_loss 0.246279 val_loss 0.249988
learning rate:  0.0001 , num:  64 , sim:  0
Epoch 11
-------------------------------
train_loss 0.107364 val_loss 0.108244
learning rate:  0.0001 , num:  64 , sim:  0
Epoch 21
-------------------------------
train_loss 0.105103 val_loss 0.106075
learning rate:  0.0001 , num:  64 , sim:  0
Epoch 31
------------------------------

In [ ]:
print(path)

In [ ]:
#net.load_state_dict(torch.load("./OU_0.pt"))
net.load_state_dict(torch.load(path))

torch.set_default_device('cpu')
print(np.min(val_error_plt))
print(np.argmin(val_error_plt))
plt.plot(np.arange(N_EPOCHS), train_error_plt, color = "r")
plt.plot(np.arange(N_EPOCHS), val_error_plt)
plt.legend(["train", "validation"], loc ="upper right")

In [ ]:
net = net.to(torch.device('cpu'))